In [1]:
import os
import pandas as pd
os.chdir("../..")

In [2]:
data = pd.read_csv('raw/affordable-homes/affordable_homes_open_data_202223')

In [3]:
# Remove un-needed columns
data.drop(columns=['LA code', 'LA name', 'District name', 'Metropolitan code', 'Metropolitan name', 'Region code', 'Region name', 'LA type 202223'], inplace=True)

In [4]:
# Rename column
data.rename(columns={'LA code 202223': 'geography_code'}, inplace=True)

In [5]:
# We are only interested in completions for now
data = data.loc[data.Completions=='Completion']

In [6]:
# Sum values per year, per local authority, per tenure type
local_authority = data.groupby(['geography_code', 'Year', 'Tenure']).sum(numeric_only=True).reset_index()

local_authority_wide = local_authority.pivot(index=['geography_code', 'Year'], columns='Tenure', values='Units')

# Add a column for all affordable homes, which is the sum of all the tenure types.
local_authority_wide['All afforable'] = local_authority_wide.sum(axis=1)

In [7]:
# Calculate the values for the whole of England
all_england = local_authority.groupby(['Year', 'Tenure']).sum(numeric_only=True).reset_index()

# Pivot to wide format
all_england_wide = all_england.pivot(index='Year', columns='Tenure', values='Units')

# Add a column for all affordable homes, which is the sum of all the tenure types.
all_england_wide['All afforable'] = all_england_wide.sum(axis=1)

# Add the geo code for England and append it to the index.
all_england_wide['geography_code'] = 'E92000001'
all_england_wide.set_index('geography_code', append=True, inplace=True)
all_england_wide = all_england_wide.reorder_levels(['geography_code', 'Year'])

In [8]:
# Concatenate the dataframes
combined = pd.concat([local_authority_wide, all_england_wide])

In [9]:
# Write the files to CSV
combined.to_csv('data/affordable-homes/by_tenure.csv')
# all_england.to_csv('data/affordable-homes/by_tenure_england.csv')

# TODO 
Use a lookup to go from LAs to met counties and regions.